# Day 13: Recurrent Neural Networks (RNNs)

## Phase 2: NLP Basics (Days 11-20)

**Estimated Time: 3-4 hours**

### Learning Objectives
- Understand sequential data and the need for recurrent architectures
- Implement vanilla RNN from scratch in NumPy
- Master backpropagation through time (BPTT)
- Understand vanishing and exploding gradient problems
- Build RNN-based language models
- Implement text generation with RNNs
- Use PyTorch's RNN modules for efficient implementation

### Prerequisites
- Day 12: Word Embeddings
- Neural network fundamentals (backpropagation)
- Linear algebra (matrix operations)
- Calculus (chain rule, gradients)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
plt.style.use('seaborn-v0_8-darkgrid')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print("Libraries loaded successfully!")

## 1. Sequential Data and Motivation

### 1.1 Why Sequences Matter

Many real-world problems involve sequential data:
- **Text**: Words in sentences, characters in documents
- **Speech**: Audio waveforms over time
- **Time series**: Stock prices, sensor readings
- **Video**: Frames over time
- **Music**: Notes in melodies

### 1.2 Limitations of Feedforward Networks

Standard feedforward networks:
- Fixed input size
- No memory of previous inputs
- Cannot capture temporal dependencies
- Process each input independently

**Example**: "The clouds are in the ___"
- Answer depends on understanding the context
- Need to remember "clouds" to predict "sky"

### 1.3 The Recurrent Idea

**Key Insight**: Use the same weights repeatedly across time steps, maintaining a hidden state that captures history.

$$h_t = f(h_{t-1}, x_t; \theta)$$

The hidden state $h_t$ encodes information from all previous inputs.

In [ ]:
# Visualizing the difference between feedforward and recurrent

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Feedforward Network
ax = axes[0]
ax.set_title('Feedforward Network', fontsize=14, fontweight='bold')

# Draw layers
layer_positions = [0.2, 0.5, 0.8]
layer_sizes = [4, 5, 3]
layer_names = ['Input', 'Hidden', 'Output']

for l, (x_pos, size, name) in enumerate(zip(layer_positions, layer_sizes, layer_names)):
    y_positions = np.linspace(0.2, 0.8, size)
    for y in y_positions:
        circle = plt.Circle((x_pos, y), 0.03, color='steelblue', alpha=0.7)
        ax.add_patch(circle)
    ax.text(x_pos, 0.05, name, ha='center', fontsize=11)
    
    # Draw connections to next layer
    if l < len(layer_positions) - 1:
        next_y = np.linspace(0.2, 0.8, layer_sizes[l+1])
        for y1 in y_positions:
            for y2 in next_y:
                ax.plot([x_pos+0.03, layer_positions[l+1]-0.03], [y1, y2], 
                       'k-', alpha=0.2, linewidth=0.5)

ax.text(0.5, 0.95, 'Fixed input size, no memory', ha='center', fontsize=11, style='italic')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

# Recurrent Network (unrolled)
ax = axes[1]
ax.set_title('Recurrent Network (Unrolled)', fontsize=14, fontweight='bold')

time_steps = 4
for t in range(time_steps):
    x_pos = 0.15 + t * 0.22
    
    # Input
    circle = plt.Circle((x_pos, 0.2), 0.04, color='lightgreen', alpha=0.8)
    ax.add_patch(circle)
    ax.text(x_pos, 0.08, f'$x_{t}$', ha='center', fontsize=10)
    
    # Hidden state
    circle = plt.Circle((x_pos, 0.5), 0.05, color='steelblue', alpha=0.8)
    ax.add_patch(circle)
    ax.text(x_pos, 0.5, f'$h_{t}$', ha='center', fontsize=10, color='white', fontweight='bold')
    
    # Output
    circle = plt.Circle((x_pos, 0.8), 0.04, color='salmon', alpha=0.8)
    ax.add_patch(circle)
    ax.text(x_pos, 0.92, f'$y_{t}$', ha='center', fontsize=10)
    
    # Vertical connections
    ax.arrow(x_pos, 0.24, 0, 0.17, head_width=0.02, head_length=0.02, fc='black')
    ax.arrow(x_pos, 0.55, 0, 0.17, head_width=0.02, head_length=0.02, fc='black')
    
    # Recurrent connection
    if t < time_steps - 1:
        ax.arrow(x_pos + 0.05, 0.5, 0.12, 0, head_width=0.02, head_length=0.02, 
                fc='red', ec='red', linewidth=2)

ax.text(0.5, 0.95, 'Shared weights, memory through hidden state', ha='center', fontsize=11, style='italic')
ax.text(0.55, 0.42, 'Recurrent\nconnection', ha='center', fontsize=9, color='red')
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.axis('off')

plt.tight_layout()
plt.show()

## 2. Vanilla RNN Architecture

### 2.1 Mathematical Formulation

At each time step $t$:

**Hidden State Update:**
$$h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)$$

**Output:**
$$y_t = W_{hy} h_t + b_y$$

**Parameters:**
- $W_{xh} \in \mathbb{R}^{H \times D}$: Input-to-hidden weights
- $W_{hh} \in \mathbb{R}^{H \times H}$: Hidden-to-hidden weights (recurrent)
- $W_{hy} \in \mathbb{R}^{V \times H}$: Hidden-to-output weights
- $b_h \in \mathbb{R}^H$: Hidden bias
- $b_y \in \mathbb{R}^V$: Output bias

Where:
- $D$: Input dimension
- $H$: Hidden dimension
- $V$: Output dimension (vocabulary size for language models)

### 2.2 Why tanh?

- Output range: $[-1, 1]$ (centered at 0)
- Prevents hidden state from exploding
- Allows both positive and negative activations
- Smoother gradients than ReLU for recurrent connections

In [ ]:
class VanillaRNN:
    """
    Vanilla RNN implementation from scratch in NumPy.
    
    Forward pass computes hidden states and outputs.
    Backward pass implements BPTT.
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        
        # Xavier initialization
        scale_xh = np.sqrt(2.0 / (input_dim + hidden_dim))
        scale_hh = np.sqrt(2.0 / (hidden_dim + hidden_dim))
        scale_hy = np.sqrt(2.0 / (hidden_dim + output_dim))
        
        self.W_xh = np.random.randn(hidden_dim, input_dim) * scale_xh
        self.W_hh = np.random.randn(hidden_dim, hidden_dim) * scale_hh
        self.W_hy = np.random.randn(output_dim, hidden_dim) * scale_hy
        
        self.b_h = np.zeros((hidden_dim, 1))
        self.b_y = np.zeros((output_dim, 1))
        
        # For storing intermediate values during forward pass
        self.cache = {}
    
    def forward(self, inputs, h_prev=None):
        """
        Forward pass through the RNN.
        
        inputs: list of input vectors [x_0, x_1, ..., x_T]
                each x_t is shape (input_dim, 1)
        h_prev: initial hidden state (hidden_dim, 1)
        
        Returns: outputs, hidden_states
        """
        if h_prev is None:
            h_prev = np.zeros((self.hidden_dim, 1))
        
        T = len(inputs)
        
        # Store for BPTT
        self.cache['inputs'] = inputs
        self.cache['h'] = {-1: h_prev}  # h_{-1} = initial state
        self.cache['y'] = {}
        
        outputs = []
        hidden_states = []
        
        h_t = h_prev
        
        for t in range(T):
            x_t = inputs[t]
            
            # Hidden state update
            # h_t = tanh(W_hh @ h_{t-1} + W_xh @ x_t + b_h)
            z_t = self.W_hh @ h_t + self.W_xh @ x_t + self.b_h
            h_t = np.tanh(z_t)
            
            # Output
            y_t = self.W_hy @ h_t + self.b_y
            
            # Store
            self.cache['h'][t] = h_t
            self.cache['y'][t] = y_t
            
            outputs.append(y_t)
            hidden_states.append(h_t)
        
        return outputs, hidden_states
    
    def softmax(self, x):
        """Numerically stable softmax."""
        exp_x = np.exp(x - np.max(x))
        return exp_x / np.sum(exp_x)
    
    def compute_loss(self, outputs, targets):
        """
        Compute cross-entropy loss.
        
        outputs: list of output vectors
        targets: list of target indices
        """
        loss = 0
        probs_list = []
        
        for t in range(len(outputs)):
            probs = self.softmax(outputs[t])
            probs_list.append(probs)
            loss += -np.log(probs[targets[t], 0] + 1e-10)
        
        self.cache['probs'] = probs_list
        return loss / len(outputs)
    
    def backward(self, targets):
        """
        Backpropagation through time (BPTT).
        
        Computes gradients for all parameters.
        """
        T = len(targets)
        
        # Initialize gradients
        dW_xh = np.zeros_like(self.W_xh)
        dW_hh = np.zeros_like(self.W_hh)
        dW_hy = np.zeros_like(self.W_hy)
        db_h = np.zeros_like(self.b_h)
        db_y = np.zeros_like(self.b_y)
        
        # Gradient of loss w.r.t. next hidden state
        dh_next = np.zeros((self.hidden_dim, 1))
        
        # Backward through time
        for t in reversed(range(T)):
            # Gradient from output layer
            # dL/dy_t = probs - one_hot(target)
            dy = self.cache['probs'][t].copy()
            dy[targets[t], 0] -= 1
            dy /= T  # Average over time steps
            
            # Gradients for output weights
            dW_hy += dy @ self.cache['h'][t].T
            db_y += dy
            
            # Gradient flowing into hidden state
            dh = self.W_hy.T @ dy + dh_next
            
            # Gradient through tanh
            # d(tanh(z))/dz = 1 - tanh^2(z)
            dz = dh * (1 - self.cache['h'][t] ** 2)
            
            # Gradients for input weights
            dW_xh += dz @ self.cache['inputs'][t].T
            db_h += dz
            
            # Gradients for recurrent weights
            dW_hh += dz @ self.cache['h'][t-1].T
            
            # Gradient to pass to previous time step
            dh_next = self.W_hh.T @ dz
        
        # Clip gradients to prevent explosion
        for grad in [dW_xh, dW_hh, dW_hy, db_h, db_y]:
            np.clip(grad, -5, 5, out=grad)
        
        return {
            'dW_xh': dW_xh,
            'dW_hh': dW_hh,
            'dW_hy': dW_hy,
            'db_h': db_h,
            'db_y': db_y
        }
    
    def update_parameters(self, grads, learning_rate):
        """Update parameters using gradients."""
        self.W_xh -= learning_rate * grads['dW_xh']
        self.W_hh -= learning_rate * grads['dW_hh']
        self.W_hy -= learning_rate * grads['dW_hy']
        self.b_h -= learning_rate * grads['db_h']
        self.b_y -= learning_rate * grads['db_y']

# Test the RNN
print("Vanilla RNN Architecture:")
print("="*50)

input_dim = 10
hidden_dim = 20
output_dim = 10
seq_length = 5

rnn = VanillaRNN(input_dim, hidden_dim, output_dim)

# Create dummy input sequence
inputs = [np.random.randn(input_dim, 1) for _ in range(seq_length)]
targets = [np.random.randint(0, output_dim) for _ in range(seq_length)]

print(f"Input dimension: {input_dim}")
print(f"Hidden dimension: {hidden_dim}")
print(f"Output dimension: {output_dim}")
print(f"Sequence length: {seq_length}")

# Forward pass
outputs, hidden_states = rnn.forward(inputs)
loss = rnn.compute_loss(outputs, targets)

print(f"\nForward pass successful!")
print(f"Loss: {loss:.4f}")
print(f"Output shape at each step: {outputs[0].shape}")
print(f"Hidden state shape at each step: {hidden_states[0].shape}")

# Backward pass
grads = rnn.backward(targets)
print(f"\nBackward pass successful!")
print(f"Gradient shapes:")
for name, grad in grads.items():
    print(f"  {name}: {grad.shape}")

## 3. Backpropagation Through Time (BPTT)

### 3.1 The Chain Rule Through Time

For a sequence of length $T$, the total loss is:
$$L = \sum_{t=1}^{T} L_t$$

The gradient of loss w.r.t. $W_{hh}$ requires summing contributions from all time steps:

$$\frac{\partial L}{\partial W_{hh}} = \sum_{t=1}^{T} \frac{\partial L_t}{\partial W_{hh}}$$

And each $\frac{\partial L_t}{\partial W_{hh}}$ involves a chain through all previous hidden states:

$$\frac{\partial L_t}{\partial W_{hh}} = \sum_{k=1}^{t} \frac{\partial L_t}{\partial h_t} \frac{\partial h_t}{\partial h_k} \frac{\partial h_k}{\partial W_{hh}}$$

### 3.2 Gradient Flow

The key term is:
$$\frac{\partial h_t}{\partial h_k} = \prod_{i=k+1}^{t} \frac{\partial h_i}{\partial h_{i-1}}$$

Each factor involves:
$$\frac{\partial h_i}{\partial h_{i-1}} = \text{diag}(1 - h_i^2) \cdot W_{hh}$$

This is a product of $(t-k)$ matrices!

In [ ]:
# Visualize BPTT computation graph

fig, ax = plt.subplots(figsize=(14, 8))

T = 4  # Time steps
spacing = 2.5

# Draw nodes
for t in range(T):
    x = t * spacing
    
    # Input
    circle = plt.Circle((x, 0), 0.3, color='lightgreen', alpha=0.8)
    ax.add_patch(circle)
    ax.text(x, 0, f'$x_{{{t}}}$', ha='center', va='center', fontsize=12)
    
    # Hidden
    circle = plt.Circle((x, 2), 0.4, color='steelblue', alpha=0.8)
    ax.add_patch(circle)
    ax.text(x, 2, f'$h_{{{t}}}$', ha='center', va='center', fontsize=12, color='white', fontweight='bold')
    
    # Output
    circle = plt.Circle((x, 4), 0.3, color='salmon', alpha=0.8)
    ax.add_patch(circle)
    ax.text(x, 4, f'$y_{{{t}}}$', ha='center', va='center', fontsize=12)
    
    # Loss
    rect = plt.Rectangle((x-0.3, 5.7), 0.6, 0.6, color='gold', alpha=0.8)
    ax.add_patch(rect)
    ax.text(x, 6, f'$L_{{{t}}}$', ha='center', va='center', fontsize=11)
    
    # Forward arrows
    ax.annotate('', xy=(x, 1.6), xytext=(x, 0.3),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    ax.annotate('', xy=(x, 3.7), xytext=(x, 2.4),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    ax.annotate('', xy=(x, 5.7), xytext=(x, 4.3),
                arrowprops=dict(arrowstyle='->', color='black', lw=1.5))
    
    # Recurrent connection
    if t < T - 1:
        ax.annotate('', xy=((t+1)*spacing-0.4, 2), xytext=(x+0.4, 2),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2))

# Backward flow arrows (dashed)
for t in range(T-1, -1, -1):
    x = t * spacing
    
    # From loss to output
    ax.annotate('', xy=(x+0.15, 4.3), xytext=(x+0.15, 5.7),
                arrowprops=dict(arrowstyle='->', color='purple', lw=1.5, linestyle='--'))
    
    # From output to hidden
    ax.annotate('', xy=(x+0.15, 2.4), xytext=(x+0.15, 3.7),
                arrowprops=dict(arrowstyle='->', color='purple', lw=1.5, linestyle='--'))
    
    # From next hidden to current (recurrent backward)
    if t < T - 1:
        ax.annotate('', xy=(x+0.4, 2.2), xytext=((t+1)*spacing-0.4, 2.2),
                    arrowprops=dict(arrowstyle='->', color='orange', lw=2, linestyle='--'))

# Labels
ax.text(-0.8, 2, '$h_{-1}$', ha='center', va='center', fontsize=11)
ax.annotate('', xy=(0-0.4, 2), xytext=(-0.5, 2),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))

# Legend
ax.plot([], [], 'r-', lw=2, label='Forward (recurrent)')
ax.plot([], [], 'k-', lw=1.5, label='Forward (vertical)')
ax.plot([], [], '--', color='purple', lw=1.5, label='Backward (vertical)')
ax.plot([], [], '--', color='orange', lw=2, label='Backward (recurrent)')
ax.legend(loc='upper right', fontsize=10)

ax.set_xlim(-1.5, (T-1)*spacing + 1.5)
ax.set_ylim(-1, 7.5)
ax.set_aspect('equal')
ax.axis('off')
ax.set_title('Backpropagation Through Time (BPTT)', fontsize=16, fontweight='bold', pad=20)

plt.tight_layout()
plt.show()

print("Key insight: Gradients flow backward through ALL previous time steps.")
print("This creates very long gradient paths, leading to vanishing/exploding gradients.")

## 4. Vanishing and Exploding Gradients

### 4.1 The Problem

Recall:
$$\frac{\partial h_t}{\partial h_k} = \prod_{i=k+1}^{t} \text{diag}(1 - h_i^2) \cdot W_{hh}$$

**If $\|W_{hh}\| < 1$**: Product shrinks exponentially → **Vanishing gradients**

**If $\|W_{hh}\| > 1$**: Product grows exponentially → **Exploding gradients**

### 4.2 Consequences

**Vanishing Gradients:**
- Cannot learn long-range dependencies
- Early time steps have negligible influence
- Network "forgets" distant past

**Exploding Gradients:**
- Numerical overflow (NaN, Inf)
- Unstable training
- Large parameter updates

### 4.3 Solutions

1. **Gradient Clipping** (for exploding)
2. **Careful initialization** (orthogonal, identity)
3. **Gated architectures** (LSTM, GRU) - Day 14
4. **Truncated BPTT** (limit backprop steps)

In [ ]:
# Demonstrate vanishing/exploding gradients

def simulate_gradient_flow(W_hh, num_steps=50):
    """
    Simulate how gradients flow backward through time.
    
    Assumes tanh activation with outputs near 0 (max gradient).
    """
    hidden_dim = W_hh.shape[0]
    
    # Start with unit gradient
    grad = np.eye(hidden_dim)
    
    gradient_norms = [np.linalg.norm(grad)]
    
    for t in range(num_steps):
        # Simplified: assume tanh derivative is 1 (near zero output)
        grad = W_hh.T @ grad
        gradient_norms.append(np.linalg.norm(grad))
    
    return gradient_norms

hidden_dim = 10
num_steps = 50

# Case 1: Vanishing (small singular values)
W_vanish = np.random.randn(hidden_dim, hidden_dim) * 0.5
# Scale to ensure max singular value < 1
W_vanish = W_vanish / (np.linalg.norm(W_vanish, 2) * 1.2)

# Case 2: Exploding (large singular values)
W_explode = np.random.randn(hidden_dim, hidden_dim) * 1.5
# Scale to ensure max singular value > 1
W_explode = W_explode / (np.linalg.norm(W_explode, 2) * 0.8)

# Case 3: Stable (orthogonal matrix)
W_stable, _ = np.linalg.qr(np.random.randn(hidden_dim, hidden_dim))

# Simulate
grads_vanish = simulate_gradient_flow(W_vanish, num_steps)
grads_explode = simulate_gradient_flow(W_explode, num_steps)
grads_stable = simulate_gradient_flow(W_stable, num_steps)

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Vanishing
axes[0].plot(grads_vanish, 'b-', linewidth=2)
axes[0].set_yscale('log')
axes[0].set_xlabel('Time Steps Back')
axes[0].set_ylabel('Gradient Norm (log scale)')
axes[0].set_title(f'Vanishing Gradients\n||W_hh||_2 = {np.linalg.norm(W_vanish, 2):.3f}')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=1e-10, color='r', linestyle='--', alpha=0.5)
axes[0].text(25, 1e-9, 'Negligible gradients', ha='center', color='r')

# Exploding
axes[1].plot(grads_explode, 'r-', linewidth=2)
axes[1].set_yscale('log')
axes[1].set_xlabel('Time Steps Back')
axes[1].set_ylabel('Gradient Norm (log scale)')
axes[1].set_title(f'Exploding Gradients\n||W_hh||_2 = {np.linalg.norm(W_explode, 2):.3f}')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=1e10, color='r', linestyle='--', alpha=0.5)
axes[1].text(25, 1e11, 'Numerical overflow!', ha='center', color='r')

# Stable
axes[2].plot(grads_stable, 'g-', linewidth=2)
axes[2].set_xlabel('Time Steps Back')
axes[2].set_ylabel('Gradient Norm')
axes[2].set_title(f'Stable Gradients (Orthogonal)\n||W_hh||_2 = {np.linalg.norm(W_stable, 2):.3f}')
axes[2].grid(True, alpha=0.3)
axes[2].set_ylim([0, 20])

plt.suptitle('Gradient Flow in RNNs', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Vanishing: After {num_steps} steps, gradient norm = {grads_vanish[-1]:.2e}")
print(f"Exploding: After {num_steps} steps, gradient norm = {grads_explode[-1]:.2e}")
print(f"Stable: After {num_steps} steps, gradient norm = {grads_stable[-1]:.2f}")

## 5. Character-Level Language Model

### 5.1 Task Description

Given a sequence of characters, predict the next character:
- Input: "hell"
- Target: "ello"

This is a **many-to-many** RNN architecture.

### 5.2 Implementation

In [ ]:
class CharRNN:
    """
    Character-level RNN Language Model.
    
    Learns to predict the next character given previous characters.
    """
    
    def __init__(self, vocab_size, hidden_dim):
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        
        # RNN
        self.rnn = VanillaRNN(vocab_size, hidden_dim, vocab_size)
        
        # For tracking
        self.losses = []
    
    def encode_char(self, char, char_to_idx):
        """One-hot encode a character."""
        vec = np.zeros((self.vocab_size, 1))
        vec[char_to_idx[char], 0] = 1
        return vec
    
    def train_step(self, input_chars, target_chars, char_to_idx, learning_rate):
        """
        Single training step on a sequence.
        """
        # Encode inputs
        inputs = [self.encode_char(c, char_to_idx) for c in input_chars]
        targets = [char_to_idx[c] for c in target_chars]
        
        # Forward pass
        outputs, _ = self.rnn.forward(inputs)
        
        # Compute loss
        loss = self.rnn.compute_loss(outputs, targets)
        
        # Backward pass
        grads = self.rnn.backward(targets)
        
        # Update parameters
        self.rnn.update_parameters(grads, learning_rate)
        
        return loss
    
    def generate(self, seed_char, char_to_idx, idx_to_char, length=100, temperature=1.0):
        """
        Generate text starting from seed character.
        
        temperature: Controls randomness (lower = more deterministic)
        """
        h = np.zeros((self.hidden_dim, 1))
        
        # Start with seed
        generated = seed_char
        x = self.encode_char(seed_char, char_to_idx)
        
        for _ in range(length):
            # Forward step
            z = self.rnn.W_hh @ h + self.rnn.W_xh @ x + self.rnn.b_h
            h = np.tanh(z)
            y = self.rnn.W_hy @ h + self.rnn.b_y
            
            # Apply temperature
            y = y / temperature
            
            # Softmax and sample
            probs = np.exp(y - np.max(y))
            probs = probs / np.sum(probs)
            
            # Sample next character
            char_idx = np.random.choice(self.vocab_size, p=probs.flatten())
            next_char = idx_to_char[char_idx]
            
            generated += next_char
            
            # Prepare next input
            x = np.zeros((self.vocab_size, 1))
            x[char_idx, 0] = 1
        
        return generated

# Prepare data
text = """
the quick brown fox jumps over the lazy dog.
a journey of a thousand miles begins with a single step.
to be or not to be that is the question.
all that glitters is not gold.
the only thing we have to fear is fear itself.
in the beginning was the word and the word was with god.
to infinity and beyond.
may the force be with you.
""".lower().strip()

# Build vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)
char_to_idx = {c: i for i, c in enumerate(chars)}
idx_to_char = {i: c for c, i in char_to_idx.items()}

print(f"Vocabulary size: {vocab_size}")
print(f"Characters: {''.join(chars)}")
print(f"Text length: {len(text)} characters")

# Initialize model
hidden_dim = 64
model = CharRNN(vocab_size, hidden_dim)

print(f"\nModel initialized with hidden_dim={hidden_dim}")

In [ ]:
# Train the character-level RNN

seq_length = 25
learning_rate = 0.1
num_iterations = 1000

print(f"Training for {num_iterations} iterations...")
print(f"Sequence length: {seq_length}")
print(f"Learning rate: {learning_rate}")
print("\n" + "="*60)

losses = []
smooth_loss = -np.log(1.0 / vocab_size) * seq_length  # Initial loss

for iteration in range(num_iterations):
    # Random starting position
    start_idx = np.random.randint(0, len(text) - seq_length - 1)
    
    # Get input and target sequences
    input_chars = text[start_idx:start_idx + seq_length]
    target_chars = text[start_idx + 1:start_idx + seq_length + 1]
    
    # Train
    loss = model.train_step(input_chars, target_chars, char_to_idx, learning_rate)
    
    # Smooth loss
    smooth_loss = 0.999 * smooth_loss + 0.001 * loss * seq_length
    losses.append(smooth_loss / seq_length)
    
    # Print progress and sample
    if (iteration + 1) % 200 == 0:
        print(f"\nIteration {iteration + 1}, Loss: {smooth_loss / seq_length:.4f}")
        
        # Generate sample
        seed = np.random.choice(list(char_to_idx.keys()))
        sample = model.generate(seed, char_to_idx, idx_to_char, length=50, temperature=0.8)
        print(f"Sample: {sample}")

print("\n" + "="*60)
print("Training complete!")

# Plot loss
plt.figure(figsize=(10, 4))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('Loss (smoothed)')
plt.title('Character-Level RNN Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Generate text with different temperatures

print("Text Generation with Different Temperatures")
print("="*60)

temperatures = [0.5, 0.8, 1.0, 1.5]
seed_char = 't'

for temp in temperatures:
    print(f"\nTemperature = {temp}:")
    generated = model.generate(seed_char, char_to_idx, idx_to_char, length=100, temperature=temp)
    print(generated)

print("\n" + "="*60)
print("Lower temperature = More conservative (picks high probability chars)")
print("Higher temperature = More random (explores lower probability chars)")

## 6. PyTorch RNN Implementation

### 6.1 Using nn.RNN

PyTorch provides optimized RNN modules with:
- Efficient CUDA kernels
- Automatic batching
- Gradient computation
- Multiple layers support

In [ ]:
class PyTorchCharRNN(nn.Module):
    """
    Character-level RNN using PyTorch.
    """
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super().__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        # Embedding layer (instead of one-hot)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # RNN
        self.rnn = nn.RNN(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True
        )
        
        # Output layer
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x, hidden=None):
        """
        x: [batch_size, seq_length] tensor of character indices
        hidden: [num_layers, batch_size, hidden_dim] hidden state
        """
        # Embed characters
        embedded = self.embedding(x)  # [batch, seq, embed_dim]
        
        # RNN forward
        output, hidden = self.rnn(embedded, hidden)  # [batch, seq, hidden]
        
        # Output layer
        logits = self.fc(output)  # [batch, seq, vocab_size]
        
        return logits, hidden
    
    def init_hidden(self, batch_size):
        """Initialize hidden state."""
        return torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)

# Prepare PyTorch data
def prepare_sequences(text, char_to_idx, seq_length):
    """Prepare input-target pairs for training."""
    inputs = []
    targets = []
    
    for i in range(0, len(text) - seq_length - 1, seq_length):
        input_seq = [char_to_idx[c] for c in text[i:i+seq_length]]
        target_seq = [char_to_idx[c] for c in text[i+1:i+seq_length+1]]
        inputs.append(input_seq)
        targets.append(target_seq)
    
    return torch.tensor(inputs), torch.tensor(targets)

# Create model
embedding_dim = 32
hidden_dim = 128
num_layers = 2

pytorch_model = PyTorchCharRNN(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_dim=hidden_dim,
    num_layers=num_layers
).to(device)

print("PyTorch Character RNN:")
print(pytorch_model)

# Count parameters
total_params = sum(p.numel() for p in pytorch_model.parameters())
print(f"\nTotal parameters: {total_params:,}")

In [ ]:
# Train PyTorch model

seq_length = 50
inputs, targets = prepare_sequences(text, char_to_idx, seq_length)
inputs = inputs.to(device)
targets = targets.to(device)

print(f"Input shape: {inputs.shape}")
print(f"Target shape: {targets.shape}")

# Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.01)

num_epochs = 200
pytorch_losses = []

print(f"\nTraining for {num_epochs} epochs...")

for epoch in range(num_epochs):
    pytorch_model.train()
    
    # Forward pass
    hidden = pytorch_model.init_hidden(inputs.size(0))
    optimizer.zero_grad()
    
    output, hidden = pytorch_model(inputs, hidden)
    
    # Reshape for loss: [batch*seq, vocab] vs [batch*seq]
    output = output.view(-1, vocab_size)
    targets_flat = targets.view(-1)
    
    loss = criterion(output, targets_flat)
    
    # Backward pass
    loss.backward()
    
    # Gradient clipping
    torch.nn.utils.clip_grad_norm_(pytorch_model.parameters(), 5)
    
    optimizer.step()
    
    pytorch_losses.append(loss.item())
    
    if (epoch + 1) % 40 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

print("Training complete!")

# Plot loss
plt.figure(figsize=(10, 4))
plt.plot(pytorch_losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('PyTorch RNN Training Loss')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Generate text with PyTorch model

def generate_pytorch(model, seed_char, char_to_idx, idx_to_char, length=100, temperature=1.0):
    """Generate text using PyTorch model."""
    model.eval()
    
    generated = seed_char
    hidden = model.init_hidden(1)
    
    # Feed seed character
    input_idx = torch.tensor([[char_to_idx[seed_char]]]).to(device)
    
    with torch.no_grad():
        for _ in range(length):
            output, hidden = model(input_idx, hidden)
            
            # Apply temperature
            output = output[:, -1, :] / temperature
            
            # Sample
            probs = torch.softmax(output, dim=-1)
            char_idx = torch.multinomial(probs, 1).item()
            
            next_char = idx_to_char[char_idx]
            generated += next_char
            
            # Prepare next input
            input_idx = torch.tensor([[char_idx]]).to(device)
    
    return generated

print("PyTorch RNN Text Generation")
print("="*60)

for temp in [0.5, 0.8, 1.0]:
    print(f"\nTemperature = {temp}:")
    sample = generate_pytorch(pytorch_model, 't', char_to_idx, idx_to_char, 
                             length=150, temperature=temp)
    print(sample)

## 7. RNN Variants and Applications

### 7.1 Sequence-to-Sequence Architectures

**Many-to-One:** Sequence Classification
- Sentiment analysis
- Document classification
- Use final hidden state

**One-to-Many:** Sequence Generation
- Image captioning
- Music generation
- Single input, multiple outputs

**Many-to-Many (Synced):** Sequence Labeling
- POS tagging
- Named Entity Recognition
- Output at each time step

**Many-to-Many (Unsynced):** Sequence Translation
- Machine translation
- Encoder-decoder architecture

In [ ]:
# Sentiment Classification (Many-to-One)

class SentimentRNN(nn.Module):
    """
    RNN for sentiment classification.
    
    Uses the final hidden state for classification.
    """
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        # x: [batch, seq_length]
        embedded = self.embedding(x)  # [batch, seq, embed_dim]
        
        # RNN
        output, hidden = self.rnn(embedded)
        # output: [batch, seq, hidden_dim]
        # hidden: [1, batch, hidden_dim]
        
        # Use last hidden state
        last_hidden = hidden.squeeze(0)  # [batch, hidden_dim]
        
        # Classify
        logits = self.fc(last_hidden)  # [batch, num_classes]
        
        return logits

# Demo
print("Sentiment Classification RNN (Many-to-One)")
sentiment_model = SentimentRNN(vocab_size=1000, embedding_dim=50, hidden_dim=64, num_classes=2)
print(sentiment_model)

# Test
batch_size = 4
seq_length = 20
sample_input = torch.randint(0, 1000, (batch_size, seq_length))
output = sentiment_model(sample_input)
print(f"\nInput shape: {sample_input.shape}")
print(f"Output shape: {output.shape} (batch_size, num_classes)")

In [ ]:
# Sequence Labeling (Many-to-Many Synced)

class SequenceLabelingRNN(nn.Module):
    """
    RNN for sequence labeling (e.g., POS tagging, NER).
    
    Outputs a label for each input token.
    """
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_tags)
    
    def forward(self, x):
        # x: [batch, seq_length]
        embedded = self.embedding(x)  # [batch, seq, embed_dim]
        
        # RNN
        output, _ = self.rnn(embedded)  # [batch, seq, hidden_dim]
        
        # Output at each time step
        logits = self.fc(output)  # [batch, seq, num_tags]
        
        return logits

# Demo
print("Sequence Labeling RNN (Many-to-Many Synced)")
labeling_model = SequenceLabelingRNN(vocab_size=1000, embedding_dim=50, hidden_dim=64, num_tags=10)
print(labeling_model)

# Test
sample_input = torch.randint(0, 1000, (batch_size, seq_length))
output = labeling_model(sample_input)
print(f"\nInput shape: {sample_input.shape}")
print(f"Output shape: {output.shape} (batch_size, seq_length, num_tags)")
print("\nEach token gets a tag distribution!")

### 7.2 Bidirectional RNN

Process sequence in both directions:
- Forward: $h_t^{\rightarrow} = f(h_{t-1}^{\rightarrow}, x_t)$
- Backward: $h_t^{\leftarrow} = f(h_{t+1}^{\leftarrow}, x_t)$
- Concatenate: $h_t = [h_t^{\rightarrow}; h_t^{\leftarrow}]$

**Advantage:** Each position has context from both past and future.

In [ ]:
# Bidirectional RNN

class BidirectionalRNN(nn.Module):
    """
    Bidirectional RNN for sequence labeling.
    """
    
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Bidirectional RNN
        self.rnn = nn.RNN(
            embedding_dim, 
            hidden_dim, 
            batch_first=True,
            bidirectional=True  # Key change!
        )
        
        # Output layer (hidden_dim * 2 because bidirectional)
        self.fc = nn.Linear(hidden_dim * 2, num_tags)
    
    def forward(self, x):
        embedded = self.embedding(x)
        
        # output shape: [batch, seq, hidden_dim * 2]
        output, _ = self.rnn(embedded)
        
        logits = self.fc(output)
        
        return logits

# Demo
print("Bidirectional RNN")
birnn_model = BidirectionalRNN(vocab_size=1000, embedding_dim=50, hidden_dim=64, num_tags=10)
print(birnn_model)

# Test
sample_input = torch.randint(0, 1000, (batch_size, seq_length))
output = birnn_model(sample_input)
print(f"\nInput shape: {sample_input.shape}")
print(f"Output shape: {output.shape}")
print("\nNotice: Each position now has information from BOTH directions!")

## 8. Practical Considerations

### 8.1 Training Tips

1. **Gradient Clipping**: Essential for RNNs
```python
torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)
```

2. **Learning Rate**: Start lower than feedforward (0.001 - 0.01)

3. **Sequence Length**: Longer sequences = more vanishing gradients

4. **Batch Size**: Smaller batches often work better

5. **Initialization**: 
   - Orthogonal for recurrent weights
   - Xavier/He for others

### 8.2 When to Use RNNs

**Use RNNs when:**
- Data is inherently sequential
- Order matters
- Variable-length inputs
- Temporal dependencies exist

**Consider alternatives when:**
- Very long sequences (>100 tokens) → LSTM/GRU or Transformers
- Parallel processing needed → Transformers
- No sequential structure → Feedforward networks

In [ ]:
# Demonstration of initialization strategies

def compare_initializations():
    """Compare different weight initialization strategies for RNNs."""
    hidden_dim = 100
    
    initializations = {
        'Random Normal': np.random.randn(hidden_dim, hidden_dim) * 0.01,
        'Xavier': np.random.randn(hidden_dim, hidden_dim) * np.sqrt(2.0 / (hidden_dim * 2)),
        'Orthogonal': np.linalg.qr(np.random.randn(hidden_dim, hidden_dim))[0],
        'Identity': np.eye(hidden_dim),
    }
    
    results = {}
    
    for name, W in initializations.items():
        # Compute max singular value
        max_sv = np.linalg.norm(W, 2)
        
        # Simulate gradient flow
        grad = np.eye(hidden_dim)
        grad_norms = [np.linalg.norm(grad)]
        
        for _ in range(50):
            grad = W.T @ grad
            grad_norms.append(np.linalg.norm(grad))
        
        results[name] = {
            'max_sv': max_sv,
            'grad_norms': grad_norms
        }
    
    return results

results = compare_initializations()

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Singular values
names = list(results.keys())
max_svs = [results[name]['max_sv'] for name in names]

axes[0].bar(names, max_svs, color=['red', 'blue', 'green', 'orange'], alpha=0.7)
axes[0].axhline(y=1.0, color='black', linestyle='--', linewidth=2, label='Ideal = 1')
axes[0].set_ylabel('Max Singular Value')
axes[0].set_title('Weight Matrix Spectral Properties')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Gradient flow
for name in names:
    axes[1].plot(results[name]['grad_norms'], label=name, linewidth=2)

axes[1].set_xlabel('Time Steps Back')
axes[1].set_ylabel('Gradient Norm')
axes[1].set_title('Gradient Flow Through Time')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_yscale('log')

plt.suptitle('RNN Weight Initialization Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Key Insight: Orthogonal initialization provides most stable gradient flow!")

## 9. Summary and Key Takeaways

### What We Learned

1. **RNNs process sequences** by maintaining a hidden state across time steps

2. **Vanilla RNN equations:**
   - $h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)$
   - $y_t = W_{hy} h_t + b_y$

3. **BPTT** computes gradients by unrolling the network through time

4. **Vanishing/Exploding Gradients** are fundamental challenges:
   - Long gradient paths cause exponential decay/growth
   - Limit learning of long-range dependencies
   - Solutions: gradient clipping, better initialization, gated units

5. **Architecture Variants:**
   - Many-to-One: Classification
   - Many-to-Many: Sequence labeling, language modeling
   - Bidirectional: Context from both directions

6. **Practical Training:**
   - Always use gradient clipping
   - Orthogonal initialization helps
   - Lower learning rates than feedforward

### Next Steps (Day 14)

- **LSTM**: Long Short-Term Memory networks
- **GRU**: Gated Recurrent Units
- These gated architectures solve the vanishing gradient problem!

In [ ]:
# Final visualization: RNN memory capability test

def test_rnn_memory(model_class, hidden_dim, seq_lengths):
    """
    Test how well RNN can remember information over different distances.
    
    Task: Remember first character and reproduce it at the end.
    """
    results = []
    
    for seq_len in seq_lengths:
        # Create simple task: copy first element to last
        model = model_class(
            vocab_size=10,
            embedding_dim=16,
            hidden_dim=hidden_dim,
            num_layers=1
        ).to(device)
        
        optimizer = optim.Adam(model.parameters(), lr=0.01)
        criterion = nn.CrossEntropyLoss()
        
        # Generate training data
        # Input: [first_char, random, random, ..., random]
        # Target: [don't care, ..., first_char]
        num_samples = 100
        
        # Train
        for epoch in range(100):
            first_chars = torch.randint(0, 10, (num_samples,)).to(device)
            inputs = torch.randint(0, 10, (num_samples, seq_len)).to(device)
            inputs[:, 0] = first_chars
            
            # Only care about last prediction
            targets = first_chars
            
            optimizer.zero_grad()
            output, _ = model(inputs, None)
            
            # Only loss on last position
            last_output = output[:, -1, :]
            loss = criterion(last_output, targets)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            first_chars = torch.randint(0, 10, (100,)).to(device)
            inputs = torch.randint(0, 10, (100, seq_len)).to(device)
            inputs[:, 0] = first_chars
            
            output, _ = model(inputs, None)
            predictions = output[:, -1, :].argmax(dim=1)
            
            accuracy = (predictions == first_chars).float().mean().item()
        
        results.append(accuracy)
    
    return results

# Test
seq_lengths = [5, 10, 20, 30, 50]
print("Testing RNN Memory Capability...")
print("Task: Remember first character and predict it at the end")
print(f"Sequence lengths: {seq_lengths}")

# This test would take time, so we'll simulate results
# In practice, you'd run: test_rnn_memory(PyTorchCharRNN, 64, seq_lengths)

# Simulated results (typical for vanilla RNN)
simulated_results = [0.95, 0.85, 0.65, 0.40, 0.15]

plt.figure(figsize=(10, 5))
plt.plot(seq_lengths, simulated_results, 'bo-', linewidth=2, markersize=10, label='Vanilla RNN')
plt.axhline(y=0.1, color='r', linestyle='--', alpha=0.5, label='Random guess (10%)')
plt.xlabel('Sequence Length (Distance to Remember)')
plt.ylabel('Accuracy')
plt.title('RNN Memory Capacity: Copying First Character to Last Position')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim([0, 1])

for i, (sl, acc) in enumerate(zip(seq_lengths, simulated_results)):
    plt.annotate(f'{acc:.0%}', (sl, acc), textcoords="offset points", 
                xytext=(0,10), ha='center')

plt.show()

print("\nKey Observation:")
print("Vanilla RNN struggles with long-range dependencies!")
print("Accuracy drops significantly as sequence length increases.")
print("\nThis motivates LSTM and GRU (Day 14) which solve this problem!")

## Exercises

### Exercise 1: Implement Truncated BPTT
Modify the VanillaRNN class to only backpropagate through a fixed number of time steps.

### Exercise 2: Name Generation
Train a character-level RNN to generate names (e.g., from a dataset of baby names).

### Exercise 3: Music Generation
Use an RNN to generate simple musical sequences (MIDI notes as tokens).

### Exercise 4: Sentiment Analysis
Implement a many-to-one RNN for sentiment classification on movie reviews.

### Exercise 5: Sequence-to-Sequence
Implement a basic encoder-decoder RNN for simple sequence-to-sequence tasks.

### Exercise 6: Gradient Analysis
Implement gradient norm tracking during training and visualize vanishing gradients.

### Exercise 7: Compare Initializations
Train RNNs with different weight initializations and compare convergence.

In [ ]:
# Starter code for Exercise 2: Name Generation

# Sample names dataset
names = [
    "emma", "olivia", "ava", "sophia", "isabella",
    "mia", "charlotte", "amelia", "harper", "evelyn",
    "liam", "noah", "william", "james", "oliver",
    "benjamin", "elijah", "lucas", "mason", "logan"
]

# Add start and end tokens
names_with_tokens = [f"^{name}$" for name in names]
print("Names with tokens:", names_with_tokens[:3])

# Build vocabulary
all_chars = set(''.join(names_with_tokens))
char_to_idx_names = {c: i for i, c in enumerate(sorted(all_chars))}
idx_to_char_names = {i: c for c, i in char_to_idx_names.items()}

print(f"\nVocabulary: {sorted(all_chars)}")
print(f"Vocabulary size: {len(all_chars)}")

# TODO: Train a character-level RNN on these names
# TODO: Generate new names by sampling from ^ token until $ is produced
print("\nExercise: Implement name generation RNN!")

## References

1. Elman, J. L. (1990). "Finding Structure in Time." Cognitive Science.
2. Rumelhart, D. E., et al. (1986). "Learning Representations by Back-Propagating Errors." Nature.
3. Bengio, Y., et al. (1994). "Learning Long-Term Dependencies with Gradient Descent is Difficult." IEEE Transactions on Neural Networks.
4. Pascanu, R., et al. (2013). "On the Difficulty of Training Recurrent Neural Networks." ICML.
5. Karpathy, A. (2015). "The Unreasonable Effectiveness of Recurrent Neural Networks." Blog post.
6. Graves, A. (2013). "Generating Sequences With Recurrent Neural Networks." arXiv.
7. Sutskever, I., et al. (2014). "Sequence to Sequence Learning with Neural Networks." NeurIPS.